In [ ]:
# Used to select random values when generating suggestions
import random

# Clue AI Class
This class contains all the functions for the clue AI. It also has a section of helper funciton, which are shown and at the bottom after all important sections

In [ ]:
class ClueAI():
  def __init__(self, cards=[]):
    # Base lists of all cards in the game
    self.people = {'Rev Green', 'Colonel Mustard', 'Professor Plum', 'Mrs. Peacock',
      'Miss Scarlett', 'Mrs. White'}
    self.weapons = {'Candlestick', 'Dagger', 'Lead Pipe', 'Revolver',
      'Rope', 'Wrench'}
    self.rooms = {'Ballroom', 'Billiard Room', 'Conservatory', 'Dining Room',
      'Hall', 'Kitchen', 'Library', 'Lounge', 'Study'}
    
    # Neighbors for each room, used to decide next room 
    self.neighbors = {
        'Ballroom': ['Conservatory', 'Billiard Room', 'Dining Room', 'Kitchen'],
        'Billiard Room': ['Library', 'Conservatory', 'Ballroom'],
        'Conservatory': ['Ballroom', 'Billiard Room'],
        'Dining Room': ['Lounge', 'Ballroom', 'Hall'],
        'Hall': ['Study', 'Library', 'Dining Room', 'Lounge'],
        'Kitchen': ['Ballroom', 'Dining Room'],
        'Library': ['Billiard Room', 'Study', 'Hall'],
        'Lounge': ['Dining Room', 'Hall'],
        'Study': ['Hall', 'Library']
    }

    # The rooms that have secret passages and where to
    self.secret_passages = {
        'Conservatory': 'Lounge',
        'Lounge': 'Conservatory',
        'Kitchen': 'Study',
        'Study': 'Kitchen'
    }

    # The starting room each player should go to
    self.start = {
      'Rev Green': 'Ballroom',
      'Colonel Mustard': 'Lounge',
      'Professor Plum': 'Study',
      'Mrs. Peacock': 'Conservatory',
      'Miss Scarlett': 'Lounge',
      'Mrs. White': 'Ballroom'
    }

    self.previous_room = 'None'
    self.room_options = set(cards) & self.rooms

    # For creating suggestions
    self.people_options = set(cards) & self.people
    self.weapon_options = set(cards) & self.weapons

    # Cards known to be in each person's hand or in the center
    self.has = {
        'P1' : set(cards.copy()),
        'P2' : set(),
        'P3' : set(),
        'P4' : set(),
        'Center' : set()
    }

    # Cards known not to be each player's hand
    self.dontHave = {
        'P2': set(),
        'P3': set(),
        'P4': set()
    }

    # Cards that are already known
    self.known = set(cards.copy()) 

    # Cards that are unknown (being with all cards and remove known)
    self.unknown = (self.people | self.weapons | self.rooms) - self.known

    # Unsolved suggestions
    self.unsolved = []

    # Solved suggestions (Only used for tracking game at end)
    self.solved = []

  # Record information from suggestion made to the find location of each card
  def suggestion(self, accuse, show, opportunity, suggest, card_shown='None'):
    # variable to store suggestion for output purposes
    suggestion = self.create_suggest_dic(accuse, show, opportunity, suggest.copy(), card_shown)

    # If accuse is player 1 and there was a card shown that was previously unknown
    if card_shown in self.unknown: 
      self.cards_known({card_shown}, show)

    # Removes all cards known in the center or in someone's hand from suggestion 
    sug = set(suggest)
    rem = set()
    for i in self.has:
      sug, r = self.remove_from_list(self.has[i], sug)
      if r != set():
        rem |= r

    # Add all cards left in sug to the don't have set of all players who had 
    # the opportunity to show a card
    for i in opportunity: 
      if i != 'P1':
        self.dontHave[i] |= sug

    # If the player showing the card did not have a card in the suggestion the 
    # agent knew the player had and P1 was not involved in this suggestion
    if (show in self.has and not self.showHas(show, rem) and show != 'P1' and accuse != 'P1'):
      sug, rem = self.remove_from_list(self.dontHave[show], sug)
      suggestion_left = {
          'suggestion': sug, 
          'player_who_shows': show
      }
      self.unsolved.append([suggestion_left, suggestion])
    else: 
      self.solved.append(suggestion)

    self.infer()

    return [self.has, self.dontHave, self.known, self.unknown, self.solved,  
           self.unsolved, len(self.has['Center']) == 3]

  # Make inferences from all information
  def infer(self):
    self.infer_from_unsolved()
    self.infer_from_dontHave()
    self.check_for_center()

  # Optimized the list of unsolved suggestions
  def infer_from_unsolved(self):
    for i in self.unsolved: 
      current = i[0]['player_who_shows']
      orig = i[0]['suggestion'].copy()
      
      # Remove from the unsolved suggestion all known cards 
      sug, rem = self.remove_from_list(self.known, i[0]['suggestion']) 

      # Remove from the suggestoin all cards we know the player who showed the 
      # card didn't have
      sug2, rem2 = self.remove_from_list(self.dontHave[current], sug)

      if len(sug2) == 1 and not self.showHas(current, orig): # We know the card
        self.cards_known(sug2, current)

      if (self.showHas(current, orig) or len(sug2) <= 1): # We solved the suggestion
        self.suggestion_solved(i)
        self.infer()
      else: # Update suggestion if it is unsolved
        i[0]['suggestion'] = sug2

  # Optimize dont have lists and learn any possible information from them
  def infer_from_dontHave(self): 
    # Any cards we know all other players don't have has to be in the center
    all_dont_have = self.dontHave_instersection()
     
    if all_dont_have != set(): 
      self.cards_known(all_dont_have, 'Center')
      self.update_options()
      self.infer()
    
    # Checks for intersection between center and dont have of all but 1 player
    if self.has['Center'] != set():
      for i in self.dontHave:
          dont_have = self.dontHave_instersection(i)
          if len(dont_have) > 0:
            print(dont_have)
            self.cards_known(dont_have, i)
            self.infer()

  # Checks if there is only one option left for card that could be in the center
  def check_for_center(self):
    cards = []
    lists = [self.people, self.weapons, self.rooms]
    for cur_list in lists: 
      if self.has['Center'] & cur_list != set(): 
        continue
      cards = cur_list - self.known
      if len(cards) == 1: 
        self.cards_known(cards, 'Center')
        self.update_options()
        self.infer()
  
  # Determines the next room to make suggestion in 
  def nextRoom(self, currentRoom, madeSuggestion, start = False, character = 'None'):
      # If the character is in the starting position, go to the closest room
      if start: 
        return self.start[character]
      
      # True if the room has in the center has been found
      roomKnown = self.has['Center'] & self.rooms != set()
    
      # Determine next room (from options if room known or unknown if not)
      curlist = self.room_options if roomKnown else self.unknown
      next = self.nextRoomHelper(currentRoom, madeSuggestion, curlist)
      
      # Set previous room to our current room if we are not still here
      self.previous_room = currentRoom if self.previous_room != currentRoom else self.previous_room
      print(self.previous_room)
      return next
 
  # Generate a suggestion for the room the AI is currently in based on the 
  # current knowledge of the system 
  def generateSuggestion(self, room):
    # Values of whether we know the room we are in or the room person and / or
    # weapon are in the center, and lists of all unknown people / weapons
    roomKnown = self.has['Center'] & self.rooms != set() or room in self.known
    weaponKnown = self.has['Center'] & self.weapons != set()
    personKnown = self.has['Center'] & self.people != set()
    unknownPeople = list(self.unknown & self.people)
    unknownWeapons = list(self.unknown & self.weapons)
    
    # Ceate a dictionary of scores for each value found in 
    scores = {}
    for i in self.dontHave: 
      scores = self.update_score(scores, self.dontHave[i] & self.unknown)
    for i in self.unsolved: 
      scores = self.update_score(scores, i[0]['suggestion'] & self.unknown)

    # Set inital vals for person, weapons, and associated scores
    person = 'None'
    weapon = 'None'
    pscore = 0 
    wscore = 0

    # Find highest Pscore and Wscore
    for i in scores:
      person, pscore = self.get_highest(i, scores[i], person, pscore, self.people)
      weapon, wscore = self.get_highest(i, scores[i], weapon, wscore, self.weapons)

    # If the person / weapon in the center is already known or we had no values 
    # that were promising for either set their values to an unknown / option card
    if personKnown or pscore == 0:
      person = self.reset_vals(personKnown, unknownPeople, list(self.people_options))
    if weaponKnown or wscore == 0:
      weapon = self.reset_vals(weaponKnown, unknownWeapons, list(self.weapon_options))
    
    # If we don't know the room and we haven't already reset a value, reset the 
    # value with the lowest score or the type with the least unknown values, or 
    # if all is equal flip a coin to choose
    if not roomKnown: 
      longer = len(unknownPeople) >= len(unknownWeapons)
      if pscore > wscore or pscore == wscore and longer:
        weapon = self.reset_vals(weaponKnown, unknownWeapons, list(self.weapon_options), True, weapon)
      elif wscore > pscore or pscore == wscore and not longer: 
        person = self.reset_vals(personKnown, unknownPeople, list(self.people_options), True, person)
    
    # Return the suggestion generated
    return [person, weapon, room]

  # Determines the card that should be shown to disprove a suggesiton
  def card_to_show(self, suggestion):
    possible_cards = set(suggestion) & self.has['P1']
    if possible_cards & self.people != set():
      return possible_cards & self.people
    elif possible_cards & self.weapons: 
      return possible_cards & self.weapons
    elif possible_cards & self.rooms: 
      return possible_cards & self.rooms
    else: 
      return 'None'

############################## Helper Functions ################################
  
  # Creates dictionary to hold inital suggestion to have a record of it
  def create_suggest_dic(self, accuse, show, opportunity, suggest, card_shown):
    sug = {
        'Accuse': accuse, 
        'Suggestion': suggest, 
        'Opportunity': opportunity,
        'Show': show, 
        'Card': card_shown
    }
    return sug

  # When we learn which player has a card, update all relevant KBs 
  def cards_known(self, card, player):
    self.has[player] = self.has[player] | card
    self.known = self.known | card
    self.unknown = self.unknown - card

    # Remove known card from list of cards players don't have
    for i in self.dontHave:
      self.dontHave[i] = self.dontHave[i] - card
  
  # Removes items in dictionary that are found in provided set suggest
  def remove_from_list(self, dic, suggest):
    removed = dic & suggest
    suggest = suggest - removed
    return suggest, removed

  # Returns true if any item in suggets is in the had of the player 'show'
  def showHas(self, show, suggest): 
    return self.overlap(self.has[show], suggest)

  # Returns true if there is an overlap bettwen set 1 and set 2
  def overlap(self, s1, s2):
    return not s1.isdisjoint(s2) 

  # Returns intersection of all don't have sets, or all don't have sets but 
  # player leftOut's set and has Center set
  def dontHave_instersection(self, leftOut='None'):
    dont_have = self.dontHave['P2'].copy() if leftOut != 'P2' else self.dontHave['P3'].copy()
    for i in self.dontHave:
      dont_have &= self.dontHave[i] if i != leftOut else dont_have
    if leftOut != 'None':
      temp = set()
      if self.overlap(self.has['Center'], self.people):
        temp |= self.people & dont_have
      if self.overlap(self.has['Center'], self.weapons):
        temp |= self.weapons & dont_have
      if self.overlap(self.has['Center'], self.rooms):
        temp |= self.rooms & dont_have
      dont_have = temp
    return dont_have

  # When a suggestion is solved add it to solved KB and remove from unsolved KB
  def suggestion_solved(self, sug):
    self.solved.append(sug[1])
    self.unsolved.remove(sug)

  # Determines the next room to go to
  def nextRoomHelper(self, currentRoom, madeSuggestion, curlist):
    # If we have not made a suggestion and the current room meets our criteria
    if not madeSuggestion and currentRoom in curlist: 
      return currentRoom

    # If the current room has a secret passage the that room meets our criteria
    if currentRoom in self.secret_passages and self.secret_passages[currentRoom] in curlist:
      return self.secret_passages[currentRoom]

    # If any neighbor meets our criteria or has a secret passage to a room that 
    # meets our criteria
    for i in self.neighbors[currentRoom]:
      if i in curlist or i in self.secret_passages and self.secret_passages[i] in curlist:
          return i
    
    # The next room we go to is a room that has not recently been visited
    nextRoom = self.neighbors[currentRoom]
    next = nextRoom[0] if nextRoom[0] != self.previous_room else nextRoom[1]
    return next
  
  # When generating a suggestion, will return a default value of either a random
  # unknown person / weapon, or a random person / weapon from their options
  def reset_vals(self, isKnown, unknown, options, loseVal = False, curval = 'None'):
    val = 'none'
    if loseVal and len(options) == 0: 
      return curval 
    curlist = options if isKnown or loseVal and len(options) > 0 else unknown
    s = random.randint(1,len(curlist)) - 1
    val = curlist[s]
    return val

  # Add 1 to the score entry for each value found to be in the current list
  def update_score(self, scores, curlist):
    for i in curlist:
      if i not in self.rooms: 
        scores[i] = 0 if i not in scores else scores[i] + 1
    return scores

  # Retruns the highest value / value pair between the two options if the 
  # current check value is in the current list
  def get_highest(self, cur, cur_score, val, val_score, curlist):
    if cur in curlist and cur_score > val_score: 
      return cur, cur_score
    return val, val_score

  # Update the option sets for to indicate which cards in in the AI's hand and 
  # in the center
  def update_options(self): 
    self.room_options |= self.rooms & self.has['Center']
    self.people_options |= self.people & self.has['Center']
    self.weapon_options |= self.weapons & self.has['Center']

# UI
This section is used to run the Clue AI, just hit start and it will explain it all (Not well commented, but also not important for AI running)

In [ ]:
people = ['Rev Green', 'Colonel Mustard', 'Professor Plum', 'Mrs. Peacock',
      'Miss Scarlett', 'Mrs. White']
weapons = ['Candlestick', 'Dagger', 'Lead Pipe', 'Revolver',
      'Rope', 'Wrench']
rooms = ['Ballroom', 'Billiard Room', 'Conservatory', 'Dining Room',
      'Hall', 'Kitchen', 'Library', 'Lounge', 'Study']
type_list = [people, weapons, rooms]
order = ['Person', 'Weapon', 'Room']

def print_list(l): 
  for iter,i in enumerate(l):
    print('{}: {}'.format(iter+1, i))

def get_input(input_type, p = True): 
  inp = input("Enter {} number:".format(input_type))
  if p:
    print()
  return int(inp)-1

def get_type_number(): 
  print("What is the card type?")
  print_list(order)
  card_type = input("Enter type number: ")
  print()
  return int(card_type)-1

def ask_for_cards():
  card_type = get_type_number()
  print_list(type_list[card_type])
  card = get_input(order[card_type],False)
  print(type_list[card_type][card],end='\n\n')
  return type_list[card_type][int(card)]

def ask_done(): 
  done = input("Are there more cards? y or n: ")
  print()
  return False if done == 'y' else True

def get_cards():
  print("Enter cards in hand")
  cards = []
  done = False
  while not done: 
    cards += [ask_for_cards()]
    done = ask_done()
  return cards

def get_suggestion():
  inp = []
  for i in range(len(order)): 
    print_list(type_list[i])
    inp += [type_list[i][get_input(order[i])]]
  return inp

def get_number_players(): 
  p = input("Enter number of players: ")
  return int(p)

def make_suggest(num): 
  sug = input("Does player {} make a suggestion? y or n: ".format(num))
  return True if sug == 'y' else False

def inc_turn(turn, players): 
  return turn + 1 if turn < players else 1

def get_show(): 
  print("Enter player number who showed the solution or enter 'None'",end='')
  show = input("if no one was able to disprove the suggestion: ")
  if show != 'None':
    show = 'P' + str(show)
  return show

def get_opp(sug, show, players): 
  opp = [] 
  i = sug
  while (True): 
    i = i + 1 if i < players else 1
    if show == i or sug == i: 
      break
    tempstr = 'P' + str(i)
    opp += [tempstr]
  return opp

def play_game():
  #players = get_number_players() # Only able to handle 4 currently
  players = 4
  # Enter player Character Name
  character = input("Enter the AI's character: ")

  cards = get_cards()
  solve = ClueAI(cards)
  rv = []
  winner = False
  turn = 1
  start = True
  atRoom = True 
  madeSuggestion = False
  currentRoom = 'None'
  while (not winner):
    if turn == 1: 
      if not atRoom: 
        print("Continue to ", currentRoom)
      else:
        next = solve.nextRoom(currentRoom, madeSuggestion, start, character)
        print("Go to", next)
      madeSuggestion = False 
      start = False
      atRoom = next == currentRoom
    print()
    sug = make_suggest(turn)
    if not sug: 
      turn = inc_turn(turn, players)
      continue 
    if turn == 1: 
      inp = solve.generateSuggestion(next) 
      print("Make suggestion ", inp)
      currentRoom = next 
      madeSuggestion = True
      atRoom = True
    else:
      inp = get_suggestion()
      print(inp[0], "with the", inp[1], "in the", inp[2])
      yorn = input("Correct? y or n: ")
      if yorn == 'n':
        continue
      if character in inp: 
        currentRoom = inp[2]
        madeSuggestion = False
        atRoom = True
    print()
    aishow = 'n'
    shown = 'None'
    show_num = 0
    if turn != 1: 
      aishow = input("Does the suggestion reach the AI? y or n: ")
    if aishow == 'y':
      shown = solve.card_to_show(inp)
      print("Show card: ", shown)
      show_num = 1
    if shown == 'None': 
      show = get_show()
      show_num = int(show[1]) if show != 'None' else 0
    opp = get_opp(turn, show_num, players)
    ac = 'P' + str(turn)

    value = 'None'
    if turn == 1:
      for iter,i in enumerate(inp): 
        print("{}: {}".format(iter+1, i))
      v = input("Enter Card Shown: ")
      value = inp[int(v) - 1] if v != 'None' else 'None'

    
    rv = solve.suggestion(ac, show, opp, inp, value)
    winner = rv[6]
    turn = inc_turn(turn, players)
    print() 
    labels = ["Player's Hands: ", "Cards not in Players Hands: ", "Known Card Locations: ", 
              "Unknown Card Locations: ", "Solved Suggestions: ", "Unsolved Suggestions: ", "Has the AI found the solution? "]
    for iter, i in enumerate(rv):
      print(labels[iter], i)

rv = play_game()

labels = ["Player's Hands: ", "Cards not in Players Hands: ", "Known Card Locations: ", 
              "Unknown Card Locations: ", "Solved Suggestions: ", "Unsolved Suggestions: ", "Has the AI found the solution? "]
for iter, i in enumerate(rv):
   print(labels[iter], i)

print()
print("Make accusaiton ", rv[0]['Center'], " to win the game.")

Enter the AI's character: Mrs. White
Enter cards in hand
What is the card type?
1: Person
2: Weapon
3: Room
Enter type number: 1

1: Rev Green
2: Colonel Mustard
3: Professor Plum
4: Mrs. Peacock
5: Miss Scarlett
6: Mrs. White
Enter Person number:1
Rev Green

Are there more cards? y or n: y

What is the card type?
1: Person
2: Weapon
3: Room
Enter type number: 2

1: Candlestick
2: Dagger
3: Lead Pipe
4: Revolver
5: Rope
6: Wrench
Enter Weapon number:2
Dagger

Are there more cards? y or n: y

What is the card type?
1: Person
2: Weapon
3: Room
Enter type number: 3

1: Ballroom
2: Billiard Room
3: Conservatory
4: Dining Room
5: Hall
6: Kitchen
7: Library
8: Lounge
9: Study
Enter Room number:9
Study

Are there more cards? y or n: n

Go to Ballroom

Does player 1 make a suggestion? y or n: y
Make suggestion  ['Professor Plum', 'Dagger', 'Ballroom']

Enter player number who showed the solution or enter 'None'if no one was able to disprove the suggestion: None
1: Professor Plum
2: Dagger
3: B

KeyboardInterrupt: ignored

# Test UI
UI used to more easily test different sections of the code. I would not recommend messing with this as it is somewhat confusing and not well documented 

In [ ]:
############################ USED ONLY FOR TESTING ##############################
# Allows faster entering of data, but makes assumptions not generally possible 
# for a game of clue, also not user friendly (Must eneter all cards as their 
# index in their respective lists, but only sometimes)
people = ['Rev Green', 'Colonel Mustard', 'Professor Plum', 'Mrs. Peacock',
      'Miss Scarlett', 'Mrs. White']
weapons = ['Candlestick', 'Dagger', 'Lead Pipe', 'Revolver',
      'Rope', 'Wrench']
rooms = ['Ballroom', 'Billiard Room', 'Conservatory', 'Dining Room',
      'Hall', 'Kitchen', 'Library', 'Lounge', 'Study']
cards = ['Miss Scarlett', 'Lead Pipe', 'Billiard Room']
#cards = ['Rev Green', 'Colonel Mustard', 'Professor Plum', 'Mrs. Peacock']
solve = ClueAI(cards)

rv = []
winner = False
while (not winner):
  for i in people: 
    print(i, ', ', solve.nextRoom('None', False, True, i))
  per = input("Enter Person: ")
  per = people[int(per)]

  weap = input("Enter Weapon: ")
  weap = weapons[int(weap)]

  room = input("Enter Room: ")
  room = rooms[int(room)]

  print(per, "with the", weap, "in the", room, "\n")
  ac = input("Enter Acuser: ")
  if ac == '1':
    ac = 'P1'
  elif ac == '2':
    ac = 'P2'
  elif ac == '3':
    ac = 'P3'
  elif ac == '4':
    ac = 'P4'

  show = input("Enter Show: ")
  if show == '1':
    show = 'P1'
  elif show == '2':
    show = 'P2'
  elif show == '3':
    show = 'P3'
  elif show == '4':
    show = 'P4'

  opp = []
  players = ['P1', 'P2', 'P3', 'P4']
  i = players.index(ac)
  while (True):
    i += 1
    if i > 3:
      i = 0
    if show == players[i] or ac == players[i]:
      break
    opp += [players[i]]

  value = 'None'

  if ac == 'P1':
    value = input("Enter Card Shown: ")
      
  rv = solve.suggestion(ac, show, opp, [per, weap, room], value)
  
  for i in rv:
    print(i)
  print("\n")
  winner = rv[6]

for i in rv:
    print(i)

Rev Green ,  Ballroom
Colonel Mustard ,  Lounge
Professor Plum ,  Study
Mrs. Peacock ,  Conservatory
Miss Scarlett ,  Lounge
Mrs. White ,  Ballroom
Enter Person: 2
Enter Weapon: 4
Enter Room: 6
Professor Plum with the Rope in the Library 

Enter Acuser: 4
Enter Show: 3
{'P1': {'Lead Pipe', 'Miss Scarlett', 'Billiard Room'}, 'P2': set(), 'P3': set(), 'P4': set(), 'Center': set()}
{'P2': {'Library', 'Rope', 'Professor Plum'}, 'P3': set(), 'P4': set()}
{'Lead Pipe', 'Miss Scarlett', 'Billiard Room'}
{'Wrench', 'Rev Green', 'Revolver', 'Rope', 'Colonel Mustard', 'Conservatory', 'Dagger', 'Dining Room', 'Mrs. White', 'Ballroom', 'Mrs. Peacock', 'Candlestick', 'Lounge', 'Study', 'Hall', 'Library', 'Professor Plum', 'Kitchen'}
[]
[[{'suggestion': {'Library', 'Rope', 'Professor Plum'}, 'player_who_shows': 'P3'}, {'Accuse': 'P4', 'Suggestion': ['Professor Plum', 'Rope', 'Library'], 'Opportunity': ['P1', 'P2'], 'Show': 'P3', 'Card': 'None'}]]
False


Rev Green ,  Ballroom
Colonel Mustard ,  Loun